# Cracking the Kaggle Competition#

# Allstate Claims Severity

[https://www.kaggle.com/c/allstate-claims-severity](https://www.kaggle.com/c/allstate-claims-severity)
<img src="./figures/allstate_front.png" width="700" />

## Evaluation Scheme
* [Mean absolute error](https://www.kaggle.com/c/allstate-claims-severity/details/evaluation)
$$\mathrm{MAE} = \frac{1}{n}\sum_1^n \lvert y_i - y_{i,0} \rvert$$
    * Different from the default evaluation/loss function that
$$\mathrm{rmse} = \sqrt{\frac{1}{n}\sum_1^n \left( y_i - y_{i,0} \right)^2}$$
    * Estimate of the median instead of the mean of the predicted target distribution

## Leaderboard Ranking
* Public Leaderboard
* Private Leaderboard
* Small improvement probably leads to HUGE boost on leaderboard

# Dataset
[https://www.kaggle.com/c/allstate-claims-severity/data](https://www.kaggle.com/c/allstate-claims-severity/data)
* csv format for both training and test datasets

In [9]:
import pandas as pd
train = pd.read_csv('./data/train.csv')
test  = pd.read_csv('./data/test.csv')
train.shape, test.shape

((188318, 132), (125546, 131))

In [10]:
test['loss'] = np.nan
df_train_test = train.append(test, ignore_index)
df_train_test.head()

,id,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,loss
0,1,A,B,A,B,A,A,A,A,B,...,0.718367,0.335060,0.30260,0.67135,0.83510,0.569745,0.594646,0.822493,0.714843,2213.18
1,2,A,B,A,A,A,A,A,A,B,...,0.438917,0.436585,0.60087,0.35127,0.43919,0.338312,0.366307,0.611431,0.304496,1283.60
2,5,A,B,A,A,B,A,A,A,B,...,0.289648,0.315545,0.27320,0.26076,0.32446,0.381398,0.373424,0.195709,0.774425,3005.09
3,10,B,B,A,B,A,A,A,A,B,...,0.440945,0.391128,0.31796,0.32128,0.44467,0.327915,0.321570,0.605077,0.602642,939.85
4,11,A,B,A,B,A,A,A,A,B,...,0.178193,0.247408,0.24564,0.22089,0.21230,0.204687,0.202213,0.246011,0.432606,2763.85


In [11]:
df_train_test.columns.values

array(['id', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7',
       'cat8', 'cat9', 'cat10', 'cat11', 'cat12', 'cat13', 'cat14',
       'cat15', 'cat16', 'cat17', 'cat18', 'cat19', 'cat20', 'cat21',
       'cat22', 'cat23', 'cat24', 'cat25', 'cat26', 'cat27', 'cat28',
       'cat29', 'cat30', 'cat31', 'cat32', 'cat33', 'cat34', 'cat35',
       'cat36', 'cat37', 'cat38', 'cat39', 'cat40', 'cat41', 'cat42',
       'cat43', 'cat44', 'cat45', 'cat46', 'cat47', 'cat48', 'cat49',
       'cat50', 'cat51', 'cat52', 'cat53', 'cat54', 'cat55', 'cat56',
       'cat57', 'cat58', 'cat59', 'cat60', 'cat61', 'cat62', 'cat63',
       'cat64', 'cat65', 'cat66', 'cat67', 'cat68', 'cat69', 'cat70',
       'cat71', 'cat72', 'cat73', 'cat74', 'cat75', 'cat76', 'cat77',
       'cat78', 'cat79', 'cat80', 'cat81', 'cat82', 'cat83', 'cat84',
       'cat85', 'cat86', 'cat87', 'cat88', 'cat89', 'cat90', 'cat91',
       'cat92', 'cat93', 'cat94', 'cat95', 'cat96', 'cat97', 'cat98',
       'cat99', 'cat100

In [12]:
features_categorical = []
features_numeric = []
for f in train.columns:
    if 'cat' in f:
        features_categorical.append(f)
    elif 'cont' in f:
        features_numeric.append(f)

## Properties of train/test dataset
* No missing value
* 116 categorical features coded by alphabets
* 14 numeric features
* 1 target column "loss"
* 188,318 claims in training data
* 125,546 claims in test data (to be predicted)

# Feature Engineering
* Limited by anonymous feature names
* "Blind" feature engineering
* Separate categorical and numeric features
* Add new features based on numerical properties of features

# Categorical Features
* Overview of the categorical features
* Processing/Encoding categorical features
* Add new categorical features

## Overview of the Categorical Features

In [13]:
cat_names = ['cat1', 'cat80', 'cat111','cat112']
for c in cat_names:
    print c+" category counts"
    print train[c].value_counts()
    print "------"

cat1 category counts
A    141550
B     46768
Name: cat1, dtype: int64
------
cat80 category counts
D    137505
B     46538
C      3492
A       783
Name: cat80, dtype: int64
------
cat111 category counts
A    128395
C     32401
E     14682
G      7039
I      3578
K      1353
M       473
O       221
Q        91
S        38
W        16
U        16
B         7
D         3
F         3
Y         2
Name: cat111, dtype: int64
------
cat112 category counts
E     25148
AH    18639
AS    17669
J     16222
AF     9368
AN     9138
N      8453
U      8356
AV     7122
AK     6726
K      6059
AI     4749
S      4201
AP     4000
G      3168
F      3149
AW     3145
A      2411
AR     2365
C      2257
O      2183
D      1645
AD     1531
AY     1414
Y      1351
AG     1331
AT     1272
AA     1241
AM     1170
AL     1130
R      1123
AX     1074
I       940
X       925
AE      834
Q       793
V       693
H       548
AO      534
T       521
L       493
W       461
AC      454
M       439
AU      434
B       

## Encoding Categorical Features
* Factorize Encoding
    * Not quite meaningful for a regression problem (especially for linear regression)
    * biased difference between categories

In [14]:
def prezero(char):
    if len(char) == 1: return '0'+char 
    else: return char

train_ = train.copy()
train_cat_factorize = pd.DataFrame()

for f in features_categorical:
    train[f] = train[f].apply(prezero)
    train_cat_factorize[f] = pd.factorize(train[f], sort=True)[0]

In [15]:
cat_names = ['cat1', 'cat113']
df = pd.DataFrame()
for c in cat_names:
    df[c] = train_[c]
    df[c+'_factorize'] = train_cat_factorize[c]
df.head()

,cat1,cat1_factorize,cat113,cat113_factorize
0,A,0,S,16
1,A,0,BM,58
2,A,0,AF,26
3,B,1,AE,25
4,A,0,BM,58


## Encoding Categorical Features
* One Hot Encoding - convert categories within one column to multiple columns of booleans of categories
    * More rational
    * Increase the dimension of the training dataset

In [16]:
from sklearn.preprocessing import OneHotEncoder

train_cat_ohe = pd.DataFrame()
ohe = OneHotEncoder(sparse=False)
tmp = ohe.fit_transform(train_cat_factorize)

for f, (i,j) in zip(features_categorical, zip(ohe.feature_indices_[0:],ohe.feature_indices_[1:])):
    cat_values = np.sort(train_[f].unique())
    for c, k in zip(cat_values, range(i,j)):
        train_cat_ohe[f+'_'+c] = tmp[:,k]

In [17]:
train_cat_dummy = pd.get_dummies(train[features_categorical], prefix=features_categorical)

In [18]:
c = 'cat80'
df = pd.DataFrame()
df[c] = train_[c]
for f in train_cat_dummy.columns:
    if c in f:
        df[f] = train_cat_dummy[f]
df.head()

,cat80,cat80_0A,cat80_0B,cat80_0C,cat80_0D
0,D,0.0,0.0,0.0,1.0
1,D,0.0,0.0,0.0,1.0
2,B,0.0,1.0,0.0,0.0
3,D,0.0,0.0,0.0,1.0
4,B,0.0,1.0,0.0,0.0


## Encoding Categorical Features

* Target Mean Encoding

[<img src="./figures/univariate_cat113.svg" width="1000" />](https://plot.ly/~zhenhou/33.embed)

* Target Mean Encoding
    * Based on target outcome (mean values) for individual categories in each feature
    * Shed light on categorical features more correlated with target
    

[<img src="./figures/univariate_cat113_target_sorted.svg" width="1000" />](https://plot.ly/~zhenhou/35.embed)

## Add New Categorical Features

# Numeric Features
* Overview of the numeric features
* Processing/Regularizing numeric features
* Add new numeric features based on feature correlation

## Overview of the numeric features
* All numeric features with values $\in [0,1]$
* Strong correlation between several pairs of features

<img src="./figures/numeric_scatter.png" width="800" />

## Processing numeric features

* BoxCox if skewness $> 0.25$

In [ ]:
from scipy.stats import boxcox, skew
from sklearn.preprocessing import StandardScaler

for f in features_numeric:
    if skew(df_train_test.loc[:num_train, f].values) > 0.25:
        df_train_test[f], lam = boxcox(df_train_test[f]+1.00)

* Standard Scaler

In [ ]:
ss = StandardScaler()
for f in features_numeric:
    df_train_test[f] = ss.fit_transform(df_train_test[f].values.reshape(-1,1).flatten())

## Add New Numeric Features
* Based on the correlation between pairs of features
* Pearson correlation map

[<img src="./figures/correlation.svg" width="800" />](https://plot.ly/~zhenhou/37.embed)

In [ ]:
cmap = df_train_test.loc[:num_train, features_numeric].corr()

# Predictive Modeling
* Cross-Validation (K-Folds)
* Gradient Boosting Machine (xgboost)
* Multi-Layer Perceptron Neural Network (keras)

## Cross-Validation
* KFolds - regression
* StratifiedKFolds - classification
* StratifiedKFolds for regression?

### RegStratifiedKFolds
* Make the target distributions of CV-train/CV-test more close between folds
* Reduce the variance of the CV-score between folds
* 23% improvement on the variance based on tests with n_folds=10

In [ ]:
from sklearn.cross_validation import _BaseKFold
from sklearn.utils.validation import check_random_state
import numpy as np

class RegStratifiedKFold(_BaseKFold):
    def __init__(self, y, n_folds=5, shuffle=False, random_state=None):
        super(RegStratifiedKFold, self).__init__(len(y), n_folds=n_folds, shuffle=shuffle, random_state=random_state)
        ylen = len(y)
        if ylen / n_folds <= 1:
            print "Too few elements in y. Still in ToDo list"
            exit()
        y_index = np.arange(ylen)[np.argsort(y)]
        num_classes = ylen / n_folds + int(ylen % n_folds != 0)
        
        num_head_tail_classes = n_folds * (ylen / n_folds / 2)
        head_classes = y_index[0:num_head_tail_classes].reshape(-1,n_folds)
        tail_classes = y_index[-num_head_tail_classes:].reshape(-1,n_folds)
        
        middle_class = y_index[num_head_tail_classes:-num_head_tail_classes] 
        middle_class = np.hstack([middle_class, -np.ones(n_folds - len(middle_class) % n_folds, dtype=int)])
        middle_class = middle_class.reshape(-1,n_folds)
        
        test_masks = np.vstack([head_classes, middle_class, tail_classes])
        self._test_masks = []
        
        if shuffle:
            rng = check_random_state(self.random_state)
            for cls in test_masks:
                rng.shuffle(cls)
                self._test_masks.append(cls.tolist())
        else:
            self._test_masks = test_masks
            
        self._test_masks = np.array(self._test_masks).T
        self._test_masks = [y[y!=-1] for y in self._test_masks]
        
    def _iter_test_masks(self):
        indarr = np.zeros( self.n, dtype = bool)
        for mask in self._test_masks:
            indarr[:] = False
            indarr[mask] = True 
            yield indarr

## XGBoost
[https://github.com/dmlc/xgboost/blob/master/doc/parameter.md](https://github.com/dmlc/xgboost/blob/master/doc/parameter.md)
* General Parameters
    * booster [default='gbtree']
    * num_boost_rounds
* Training Parameters
    * objective [default='reg:linear']
    * eval_metric
    * early_stopping_rounds
    * seed
* Model Parameters
    * max_depth [default=6]
    * min_child_weight [default=1]
    * gamma [default=1]
    * colsample_bytree [default=1]
    * subsample [default=1]
    * alpha [default=0]
    * lambda [default=1]
    * eta [default=0.3]

### First XGBoost Run

In [ ]:
import os
import pandas as pd
import cPickle as pkl

import xgboost
from sklearn.cross_validation import KFold

run_type = 'MAE+ContReg+CatCoadd'

train_file = './data/train_'+run_type+'.hdf5'
test_file  = './data/test_'+run_type+'.hdf5'

train  = pd.read_hdf(train_file)
test   = pd.read_hdf(test_file)
target = train['loss'].values
train.drop('loss', axis=1, inplace=True)

params = {'objective':'reg:linear',
          'booster': 'gbtree',
          'eval_metric': 'mae',
          'max_depth': 10,
          'seed': 2016,
          'eta':0.03}

evals_result = []

kf = KFold(train.shape[0], n_folds=5, shuffle=True, random_state=2016)
for i, (train_index, test_index) in enumerate(kf):
    train_target_fold = target[train_index]
    test_target_fold  = target[test_index]

    dtrain   = xgboost.DMatrix(train.iloc[train_index], label=train_target_fold)
    dtest_cv = xgboost.DMatrix(train.iloc[test_index], label=test_target_fold)

    evals = [(dtrain,'train'), (dtest_cv,'test')] 
    evals_result_fold = {}

    xgb_train = xgboost.train(params, dtrain, num_boost_round=5000, evals=evals, evals_result=evals_result_fold)
    evals_result.append(evals_result_fold)

with open('./outputs/first_xgboost_5_KFolds.pkl', 'wb') as fp:
    pkl.dump(evals_result, fp)

[<img src="./figures/first_xgboost.svg" width="800" />](https://plot.ly/~zhenhou/39.embed)

* Over-fitting
* Improvements? Optimizations?

### Objective/Loss Function
* XGBoost default = rmse
* Customized objective function for XGBoost
    * [Examples of loss functions for regression](http://research.microsoft.com/en-us/um/people/zhang/INRIA/Publis/Tutorial-Estim/node24.html)
        * "Fair" function
            $$\rho(x) = c^2\left[\frac{\lvert x \rvert}{c} - \log (1+\frac{\lvert x \rvert}{c})\right]$$
            * Numerical stability
            * Nicely converging procedures
            * One free parameter $c$

### Target Transformation
* Target (insurance claim) distribution in training dataset

[<img src="./figures/target.svg" width="1000" />](https://plot.ly/~zhenhou/20.embed)

* $y \rightarrow y' = \log(y+a)$
* $y \rightarrow y' = y^b$

[<img src="./figures/target_transformed.svg" width="1000" />](https://plot.ly/~zhenhou/22.embed)

* We choose logarithm transformation with one free parameter $a$.

### Parameter tuning
[http://xgboost.readthedocs.io/en/latest/how_to/param_tuning.html](http://xgboost.readthedocs.io/en/latest/how_to/param_tuning.html)
* RegStratifiedKfold with n_folds = 10
* $\{a, \ c, \ \mathrm{max\_depth},\ \mathrm{min\_child\_weight}, \ \mathrm{gamma}, \ \mathrm{colsample\_bytree}, \ \mathrm{subsample}, \ \mathrm{alpha}, \ \mathrm{lambda}, \ \mathrm{eta}\}$
* 10 sub-models to tune with $\mathrm{max\_depth} = \{8,9,10,11,12\}$ and $\mathrm{gamma} = \{0,1\}$
* Tuning other 8 parameters for each sub-model
* Hierarchical tuning scheme
    * $\{a, \ c, \ \mathrm{min\_child\_weight}, \ \mathrm{colsample\_bytree}\}$
    * $\{\mathrm{subsample}, \ \mathrm{alpha}, \ \mathrm{lambda}\}$
    * reduce $\mathrm{eta}$

### Parameter tuning
* Stage0 - Very beginning
* Stage1 - Coarse grid search in $\{a, \ c, \ \mathrm{min\_child\_weight}, \ \mathrm{colsample\_bytree}\}$
* Stage2 - Finer grid search in $\{a, \ c, \ \mathrm{min\_child\_weight}, \ \mathrm{colsample\_bytree}\}$
* Stage3 - Further finer grid search in $\{a, \ c, \ \mathrm{min\_child\_weight}, \ \mathrm{subsample}\}$
* Stage4 - Final grid search in $\{\mathrm{alpha}, \ \mathrm{lambda}\}$
* Stage5 - Reduce $\mathrm{eta}$
* Stage6 - Linear combination for each $\mathrm{max\_depth}$

[<img src="./figures/parameter_tuning_xgboost.svg" width="1000" />](https://plot.ly/~zhenhou/41.embed)

## Multi-Layer Perceptron (MLP)

# Ensemble Models

# Improvements beyond the Default

* Target encoding categorical features (XGBoost)
* Adding new features, both categorical and numeric
* Stratified KFolds for regression
* Customized objective function with one free parameter
* Target transformation with one free parameter
* XGBoost parameters tuning along with two additional nuisance parameters (above)
* Bagging/Bootstrapping for MLP

# Things not quite helpful

* One Hot Encoding for XGBoost
* Target encoding categorical features for MLP
* PCA numeric features for XGBoost
* PCA encoded categorical features for XGBoost